# Import Libraries

In [ ]:
import os
import sys
from tqdm import tqdm
import random
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import pickle

import shutil
from matplotlib import pyplot as plt

import warnings; warnings.simplefilter('ignore')

pd.options.display.max_columns = None
pd.options.display.max_rows = 30
np.set_printoptions(threshold=sys.maxsize)

# Dry condition routing

In [ ]:
dry_mean_traveltime_df = pd.DataFrame(columns=['net_id','dry_mean_time_s'], index=range(4190))
counter = 0

for graph_file in tqdm(os.listdir('L:yiyi/graphs_cov_no_res3/')):
    
    G1 =  pickle.load(open('L:yiyi/graphs_cov_no_res3/'+graph_file, 'rb'))
    net_id = graph_file[15:][:-3]
    
    try:
        # Add speed and travel_time on each graph edge
        G1_speed = ox.speed.add_edge_speeds(G1)
        G1_speed_time = ox.speed.add_edge_travel_times(G1_speed)

        # Load OD probability list
        OD_probability_1 = pd.read_csv(r'L:\yiyi\ODpts_propbability\n_4229_bnd_' + str(int(net_id)-1) + '.txt')
        pt_id_lst = list(OD_probability_1['pointid'])
        pt_prob_lst = list(OD_probability_1['grid_code'])

        OD_df = pd.DataFrame(columns=['index',
                                      'O_node_id',
                                      'D_node_id',
                                      'O_lat',
                                      'O_lon',
                                      'D_lat',
                                      'D_lon',
                                      'OD_route',
                                      'OD_route_length_m',
                                      'OD_route_time_s'])
        index = 0
        epsilon = float('inf')
        travel_time_lst = []

        while epsilon > 0.01:

            #Sample a point based on probability
            sample_O_pt = random.choices(pt_id_lst, pt_prob_lst)[0]
            sample_D_pt = random.choices(pt_id_lst, pt_prob_lst)[0]
            while sample_O_pt == sample_D_pt:
                sample_D_pt = random.choices(pt_id_lst, pt_prob_lst)[0]

            # O
            sample_O_lat = OD_probability_1.loc[OD_probability_1['pointid'] == sample_O_pt, 'POINT_X'].iloc[0]
            sample_O_lon = OD_probability_1.loc[OD_probability_1['pointid'] == sample_O_pt, 'POINT_Y'].iloc[0]
            # D
            sample_D_lat = OD_probability_1.loc[OD_probability_1['pointid'] == sample_D_pt, 'POINT_X'].iloc[0]
            sample_D_lon = OD_probability_1.loc[OD_probability_1['pointid'] == sample_D_pt, 'POINT_Y'].iloc[0]

            O_node_id = ox.distance.get_nearest_node(G1, (sample_O_lon, sample_O_lat), method='haversine') # lon, lat
            D_node_id = ox.distance.get_nearest_node(G1, (sample_D_lon, sample_D_lat), method='haversine')

            # Routing
            try:
                OD_route = nx.shortest_path(G1_speed_time, O_node_id, D_node_id, weight='travel_time')
            except:
                continue

            OD_route_length = sum(ox.utils_graph.get_route_edge_attributes(G1_speed_time, OD_route, 'length')) # meters
            OD_route_time =sum(ox.utils_graph.get_route_edge_attributes(G1_speed_time, OD_route, 'travel_time')) # seconds


            OD_df.loc[index] = [index+1, O_node_id, D_node_id, sample_O_lat, sample_O_lon, sample_D_lat, sample_D_lon,
                            OD_route, OD_route_length, OD_route_time]

            travel_time_lst.append(OD_route_time)

            if len(travel_time_lst) == 1:
                epsilon = float('inf')
            else:
                epsilon = abs(np.mean(travel_time_lst) - np.mean(travel_time_lst[:-1]))

            index += 1
            OD_df.to_csv('L:/yiyi/dry_OD_routing/G_' + net_id + '_dry.csv')
            
        dry_mean_traveltime_df.at[counter, 'net_id'] = net_id
        dry_mean_traveltime_df.at[counter, 'dry_mean_time_s'] = np.mean(travel_time_lst)
    
        counter += 1
    except:
        print(net_id)

# Wet condition routing

In [ ]:
# Prepare graphs to process
RP_lst = [5, 10, 20, 50, 75, 100, 200, 250, 500, 1000]
flood_graphs_lst = []
for rp in RP_lst:
    one_lst = []
    for file in os.listdir('L:/yiyi/2576_disrupted_graphs_30mm/FUP_' + str(rp) + '/'):
        net_id = int(file[2:][:(-18) - len(str(rp))])
        one_lst.append(net_id)
    flood_graphs_lst.append(one_lst)
    
common_graphs = set(flood_graphs_lst[0])
for lst in flood_graphs_lst[1:]:
    common_graphs.intersection_update(lst)
print('Number of common graphs:',len(common_graphs))

RP_lst = [5, 10, 20, 50, 75, 100, 200, 250, 500, 1000]

dry_routing_dir = 'L:/yiyi/dry_OD_routing/'

# for file in os.listdir(dry_routing_dir):
for net in tqdm(common_graphs):
    dry_OD_df = pd.read_csv(dry_routing_dir + 'G_' + str(net) + '_dry.csv', index_col=0)
    net_id = net
    wet_OD_df = dry_OD_df.copy()
    # Initiate flooded columns
    for rp in RP_lst:
        wet_OD_df['OD_route_FUP_' + str(rp)] = None
        wet_OD_df['OD_length_FUP_' + str(rp)] = 0
        wet_OD_df['OD_time_FUP_' + str(rp)] = 0
    for rp in RP_lst:
        graph_disrupted = pickle.load(open('L:/yiyi/2576_disrupted_graphs_30mm/FUP_'+ str(rp) + '/G_' + 
                                               str(net_id) + '_disrupted_FUP_' + str(rp) + '.pk', 'rb'))
        print(rp)
        for row_idx in range(wet_OD_df.shape[0]):
            O_node_id = wet_OD_df.iloc[row_idx]['O_node_id']
            D_node_id = wet_OD_df.iloc[row_idx]['D_node_id']
            try:
                wet_OD_route = nx.shortest_path(graph_disrupted, O_node_id, D_node_id, weight='travel_time')
                wet_OD_route_length = sum(ox.utils_graph.get_route_edge_attributes(graph_disrupted, wet_OD_route, 'length')) # meters
                wet_OD_route_time =sum(ox.utils_graph.get_route_edge_attributes(graph_disrupted, wet_OD_route, 'travel_time')) # seconds

                wet_OD_df.at[row_idx, 'OD_route_FUP_' + str(rp)] = wet_OD_route
                wet_OD_df.at[row_idx, 'OD_length_FUP_' + str(rp)] = wet_OD_route_length
                wet_OD_df.at[row_idx, 'OD_time_FUP_' + str(rp)] = wet_OD_route_time
                
            except:
                continue
    wet_OD_df.to_csv('L:yiyi/2576_dry_wet_OD_routing/G_' + str(net) + '_dry_wet_routing.csv')